In [11]:
import os
from collections import defaultdict

sample_type='wood'
for i in ["gpx","sod","rec","lex","umu"]:
    gene_suffix=i
    # 文件路径（需要根据你的实际路径修正）
    stat_file = "/storage/jufengLab/luogaoyang/metagenome_project/DSR/1_clean/nonbio_all_reads/BIN_REFINEMENT_1/metawrap_50_10_bins.stats"
    lex_dir =os.path.join("/storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation",sample_type+"_bin","ROS_gene",gene_suffix) 

    # 读取bin的长度信息
    bin_lengths = {}
    with open(stat_file, 'r') as f:
        # 跳过表头
        next(f)
        for line in f:
            parts = line.strip().split("\t")  # 使用tab作为分隔符
            bin_id = parts[0]  # bin ID在第1列
            bin_length = int(parts[6])  # bin长度在第7列（size）
            bin_lengths[bin_id] = bin_length

    # 统计每个bin的gene copy数
    gene_copy_counts = defaultdict(int)

    # 遍历lex目录下的所有文件
    for file_name in os.listdir(lex_dir):
        file_path = os.path.join(lex_dir, file_name)
        
        # 读取每个文件，统计基因拷贝数
        with open(file_path, 'r') as f:
            unique_queries = set()  # 用于存储唯一的query
            for line in f:
                parts = line.strip().split("\t")  # 假设MMseq输出文件是用tab分隔的
                query_id = parts[0]  # 第一列是query
                unique_queries.add(query_id)  # 将query_id加入set，自动去重
            
            # 统计每个文件的基因拷贝数
            bin_id = file_name.split('_')[0]  # 假设bin ID是文件名的一部分，形如bin_XXX_rec.m8
            gene_copy_counts[bin_id] += len(unique_queries)  # 统计唯一query的数量

    # 生成输出结果
    output = []
    for bin_id, copy_count in gene_copy_counts.items():
        bin_length = bin_lengths.get(bin_id, 0)  # 获取bin长度，默认0
        if bin_length > 0:
            copy_per_length = copy_count / bin_length * 1000000
        else:
            copy_per_length = 0  # 避免除以0的错误
        output.append([bin_id, copy_count, bin_length, copy_per_length])

    # 输出结果到文件
    output_file = os.path.join("/storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation",sample_type+"_bin","ROS_gene",gene_suffix+"_gene_copy_stats.txt")
    with open(output_file, 'w') as f:
        f.write("Bin_ID\tCopy_Count\tBin_Length\tCopy_per_Length\n")
        for row in output:
            f.write("\t".join(map(str, row)) + "\n")

    print(f"结果已保存到 {output_file}")

结果已保存到 /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/wood_bin/ROS_gene/gpx_gene_copy_stats.txt
结果已保存到 /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/wood_bin/ROS_gene/sod_gene_copy_stats.txt
结果已保存到 /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/wood_bin/ROS_gene/rec_gene_copy_stats.txt
结果已保存到 /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/wood_bin/ROS_gene/lex_gene_copy_stats.txt
结果已保存到 /storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/wood_bin/ROS_gene/umu_gene_copy_stats.txt


In [ ]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# 文件路径
bio_rec_path = "/storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/bio_bin/ROS_gene/rec_gene_copy_stats.txt"
nonbio_rec_path = "/storage/jufengLab/luogaoyang/metagenome_project/DSR/4_annotation/nonbio_bin/ROS_gene/rec_gene_copy_stats.txt"

# 读取数据
bio_rec = pd.read_csv(bio_rec_path, sep="\t")
nonbio_rec = pd.read_csv(nonbio_rec_path, sep="\t")

# 提取 Copy_per_Length 列的数据
bio_values = bio_rec['Copy_per_Length']
nonbio_values = nonbio_rec['Copy_per_Length']

# 1. 正态性检验
bio_normal_test = stats.shapiro(bio_values)
nonbio_normal_test = stats.shapiro(nonbio_values)
print(f"bio 正态性检验 p-value: {bio_normal_test.pvalue}")
print(f"nonbio 正态性检验 p-value: {nonbio_normal_test.pvalue}")

# 根据正态性检验结果决定使用何种检验方法
if bio_normal_test.pvalue > 0.05 and nonbio_normal_test.pvalue > 0.05:
    # 如果两个组都符合正态分布，使用 t 检验
    t_stat, t_pvalue = stats.ttest_ind(bio_values, nonbio_values)
    print(f"T检验结果: t-statistic = {t_stat}, p-value = {t_pvalue}")
else:
    # 如果不符合正态分布，使用 Mann-Whitney U检验
    u_stat, u_pvalue = stats.mannwhitneyu(bio_values, nonbio_values, alternative='two-sided')
    print(f"Mann-Whitney U 检验结果: U-statistic = {u_stat}, p-value = {u_pvalue}")

# 绘制箱线图
plt.figure(figsize=(8, 6))
sns.boxplot(x=['bio'] * len(bio_values) + ['nonbio'] * len(nonbio_values),
            y=bio_values.tolist() + nonbio_values.tolist())
sns.swarmplot(x=['bio'] * len(bio_values) + ['nonbio'] * len(nonbio_values),
              y=bio_values.tolist() + nonbio_values.tolist(), color=".25", jitter=True)

# 设置图形标题和标签
plt.title('Comparison of rec gene between bio and nonbio groups')
plt.xlabel('Group')
plt.ylabel('Copy per Length')

# 显示图形
plt.show()